# 4.4 — Temporal Fusion Transformer: Residual Load
Point predictions with interpretable attention. 24h ahead, trained 2015–2017, tested 2018.

Residual load = total load - wind onshore - solar. No TSO forecast for this derived target.

In [ ]:
import pandas as pd
import numpy as np
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

df = pd.read_parquet('../cleaned_data.parquet')
df['time'] = pd.to_datetime(df['time'], utc=True)

# Compute residual load
df['residual_load'] = df['total load actual'] - df['generation wind onshore'] - df['generation solar']

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
try:
    t = torch.randn(2, 2, device=device)
    _ = t @ t
except:
    device = torch.device('cpu')

print(f"Shape: {df.shape}")
print(f"PyTorch: {torch.__version__}")
print(f"Device: {device}")

Prepare features and normalize using training stats

In [ ]:
target_col = 'residual_load'

weather_cols = [
    'wind_speed_madrid', 'wind_speed_bilbao', 'wind_speed_barcelona',
    'wind_speed_seville', 'wind_speed_valencia',
    'pressure_madrid', 'pressure_bilbao', 'pressure_barcelona',
    'pressure_seville', 'pressure_valencia',
    'clouds_all_madrid', 'clouds_all_bilbao', 'clouds_all_barcelona',
    'clouds_all_seville', 'clouds_all_valencia',
    'temp_madrid', 'temp_bilbao', 'temp_barcelona',
    'temp_seville', 'temp_valencia',
]
time_cols = ['hour', 'month']
feature_cols = weather_cols + time_cols

# Normalize using training data stats only
train_mask = df['time'].dt.year <= 2017

target_mean = df.loc[train_mask, target_col].mean()
target_std = df.loc[train_mask, target_col].std()

feat_means = df.loc[train_mask, feature_cols].mean()
feat_stds = df.loc[train_mask, feature_cols].std().replace(0, 1)

target_norm = (df[target_col].values - target_mean) / target_std
features_norm = ((df[feature_cols] - feat_means) / feat_stds).fillna(0).values

# Combine: [target, features] as input channels
all_data = np.column_stack([target_norm, features_norm]).astype(np.float32)

print(f"Input channels: {all_data.shape[1]} (1 target + {len(feature_cols)} features)")
print(f"Residual load mean: {target_mean:.0f} MW, std: {target_std:.0f} MW")

Sliding window dataset — 168h context, 24h prediction

In [ ]:
context_length = 168   # 7 days of history
prediction_length = 24  # 24h ahead

class TimeSeriesDataset(Dataset):
    def __init__(self, data, ctx_len, pred_len, start_idx, end_idx):
        self.data = data
        self.ctx_len = ctx_len
        self.pred_len = pred_len
        self.start = start_idx
        self.end = end_idx

    def __len__(self):
        return self.end - self.start - self.ctx_len - self.pred_len + 1

    def __getitem__(self, idx):
        i = self.start + idx
        x = self.data[i : i + self.ctx_len]
        y = self.data[i + self.ctx_len : i + self.ctx_len + self.pred_len, 0]
        x_future = self.data[i + self.ctx_len : i + self.ctx_len + self.pred_len, 1:]
        return (
            torch.from_numpy(x),
            torch.from_numpy(x_future),
            torch.from_numpy(y),
        )

train_end = int(train_mask.sum())
train_ds = TimeSeriesDataset(all_data, context_length, prediction_length, 0, train_end)
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=0)

print(f"Train samples: {len(train_ds)}")
print(f"Context: {context_length}h, Prediction: {prediction_length}h")

TFT model — variable selection, gated residual networks, LSTM, interpretable multi-head attention

In [ ]:
class GatedResidualNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.gate_fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(output_size)
        self.skip = nn.Linear(input_size, output_size) if input_size != output_size else nn.Identity()

    def forward(self, x):
        residual = self.skip(x)
        h = F.elu(self.fc1(x))
        h = self.dropout(h)
        output = self.fc2(h)
        gate = torch.sigmoid(self.gate_fc(h))
        return self.layer_norm(gate * output + residual)


class VariableSelectionNetwork(nn.Module):
    def __init__(self, num_vars, d_model, dropout=0.1):
        super().__init__()
        self.num_vars = num_vars
        self.d_model = d_model
        self.var_transforms = nn.ModuleList([nn.Linear(1, d_model) for _ in range(num_vars)])
        self.weight_network = GatedResidualNetwork(num_vars * d_model, d_model, num_vars, dropout)

    def forward(self, x):
        var_outputs = []
        for i in range(self.num_vars):
            var_outputs.append(self.var_transforms[i](x[:, :, i:i+1]))
        var_stack = torch.stack(var_outputs, dim=2)
        flat = var_stack.reshape(x.shape[0], x.shape[1], -1)
        weights = F.softmax(self.weight_network(flat), dim=-1)
        selected = (var_stack * weights.unsqueeze(-1)).sum(dim=2)
        return selected, weights


class InterpretableMultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, self.d_k)
        self.out_proj = nn.Linear(self.d_k, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):
        bs = q.size(0)
        Q = self.W_q(q).view(bs, -1, self.n_heads, self.d_k).transpose(1, 2)
        K = self.W_k(k).view(bs, -1, self.n_heads, self.d_k).transpose(1, 2)
        V = self.W_v(v).unsqueeze(1).expand(-1, self.n_heads, -1, -1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.d_k ** 0.5)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        context = torch.matmul(attn, V)
        context = context.mean(dim=1)
        return self.out_proj(context), attn


class TemporalFusionTransformer(nn.Module):
    def __init__(self, num_observed, num_known_future, d_model=32, n_heads=4,
                 n_lstm_layers=1, pred_len=24, dropout=0.1):
        super().__init__()
        self.pred_len = pred_len
        self.d_model = d_model
        self.obs_vsn = VariableSelectionNetwork(num_observed, d_model, dropout)
        self.fut_vsn = VariableSelectionNetwork(num_known_future, d_model, dropout)
        self.encoder_lstm = nn.LSTM(d_model, d_model, n_lstm_layers, batch_first=True,
                                     dropout=dropout if n_lstm_layers > 1 else 0)
        self.decoder_lstm = nn.LSTM(d_model, d_model, n_lstm_layers, batch_first=True,
                                     dropout=dropout if n_lstm_layers > 1 else 0)
        self.lstm_gate = GatedResidualNetwork(d_model, d_model, d_model, dropout)
        self.attention = InterpretableMultiHeadAttention(d_model, n_heads, dropout)
        self.attn_gate = GatedResidualNetwork(d_model, d_model, d_model, dropout)
        self.output_proj = nn.Linear(d_model, 1)

    def forward(self, x_observed, x_future):
        enc_selected, enc_weights = self.obs_vsn(x_observed)
        dec_selected, dec_weights = self.fut_vsn(x_future)
        enc_out, (h, c) = self.encoder_lstm(enc_selected)
        dec_out, _ = self.decoder_lstm(dec_selected, (h, c))
        lstm_out = torch.cat([enc_out, dec_out], dim=1)
        input_cat = torch.cat([enc_selected, dec_selected], dim=1)
        lstm_out = self.lstm_gate(lstm_out) + input_cat
        attn_out, attn_weights = self.attention(lstm_out, lstm_out, lstm_out)
        attn_out = self.attn_gate(attn_out) + lstm_out
        decoder_out = attn_out[:, -self.pred_len:, :]
        output = self.output_proj(decoder_out).squeeze(-1)
        return output, enc_weights, attn_weights


num_observed = all_data.shape[1]      # target + all features
num_known_future = len(feature_cols)  # features only (no target in future)

model = TemporalFusionTransformer(
    num_observed=num_observed,
    num_known_future=num_known_future,
    d_model=32,
    n_heads=4,
    n_lstm_layers=1,
    pred_len=prediction_length,
    dropout=0.1,
).to(device)

print(f"TFT parameters: {sum(p.numel() for p in model.parameters()):,}")

Train with MSE loss, cosine annealing LR

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
criterion = nn.MSELoss()

n_epochs = 30
for epoch in range(n_epochs):
    model.train()
    losses = []
    for x_ctx, x_fut, y in train_loader:
        x_ctx, x_fut, y = x_ctx.to(device), x_fut.to(device), y.to(device)
        preds, _, _ = model(x_ctx, x_fut)
        loss = criterion(preds, y)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        losses.append(loss.item())

    scheduler.step()
    avg_loss = np.mean(losses)
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")

print('Training complete')

Generate 24h-ahead forecasts on 2018 test set

In [ ]:
model.eval()
test_start = train_end
test_end = len(all_data)

all_preds = []
all_actuals = []
all_times = []

with torch.no_grad():
    for i in range(test_start, test_end - prediction_length, prediction_length):
        if i - context_length < 0:
            continue

        x_ctx = torch.from_numpy(all_data[i - context_length : i]).unsqueeze(0).to(device)
        x_fut = torch.from_numpy(all_data[i : i + prediction_length, 1:]).unsqueeze(0).to(device)

        preds, _, _ = model(x_ctx, x_fut)

        # Denormalize
        pred_mw = preds.squeeze().cpu().numpy() * target_std + target_mean
        actual_mw = all_data[i : i + prediction_length, 0] * target_std + target_mean
        times = df['time'].iloc[i : i + prediction_length].values

        all_preds.append(pred_mw)
        all_actuals.append(actual_mw)
        all_times.append(times)

print(f"Generated {len(all_preds)} forecast windows across 2018")

Evaluate: MAE, RMSE, MAPE (no TSO baseline for residual load)

In [ ]:
# Flatten all windows for aggregate metrics
flat_preds = np.concatenate(all_preds)
flat_actuals = np.concatenate(all_actuals)

# TFT metrics
tft_mae = np.mean(np.abs(flat_actuals - flat_preds))
tft_rmse = np.sqrt(np.mean((flat_actuals - flat_preds) ** 2))
tft_mape = np.mean(np.abs((flat_actuals - flat_preds) / np.clip(flat_actuals, 1, None))) * 100

print(f"{'Metric':<10} {'TFT':>10}")
print('-' * 22)
print(f"{'MAE (MW)':<10} {tft_mae:>10.1f}")
print(f"{'RMSE (MW)':<10} {tft_rmse:>10.1f}")
print(f"{'MAPE (%)':<10} {tft_mape:>10.1f}")
print("\n(No TSO forecast available for residual load)")

# Per-hour MAE across the 24h forecast horizon
per_hour_mae = np.zeros(prediction_length)
for h in range(prediction_length):
    h_preds = np.array([p[h] for p in all_preds])
    h_actuals = np.array([a[h] for a in all_actuals])
    per_hour_mae[h] = np.mean(np.abs(h_actuals - h_preds))

print(f"\nPer-hour MAE (MW):")
print(f"  h+1:  {per_hour_mae[0]:.0f}  |  h+6:  {per_hour_mae[5]:.0f}  |  h+12: {per_hour_mae[11]:.0f}  |  h+24: {per_hour_mae[23]:.0f}")

Predicted vs actual — sample week

In [ ]:
sample_windows = range(9, 16)

sample_pred = []
sample_actual = []
sample_time = []

for w in sample_windows:
    if w >= len(all_preds):
        break
    sample_pred.extend(all_preds[w])
    sample_actual.extend(all_actuals[w])
    sample_time.extend(pd.to_datetime(all_times[w]))

fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(sample_time, sample_actual, color='#2c3e50', linewidth=1.5, label='Actual')
ax.plot(sample_time, sample_pred, color='coral', linewidth=1.3, label='TFT Predicted')
ax.set_ylabel('MW')
ax.set_title('TFT Residual Load')
ax.legend()
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

Export JSON for dashboard

In [ ]:
import os
os.makedirs('../dashboard/public/data', exist_ok=True)

# Build sample week data
sample_data = []
for w in range(9, 16):
    if w >= len(all_preds):
        break
    pred = all_preds[w]
    actual = all_actuals[w]
    times = all_times[w]
    for h in range(prediction_length):
        t = pd.Timestamp(times[h])
        sample_data.append({
            'time': t.strftime('%Y-%m-%d %H:%M'),
            'actual': round(float(actual[h]), 1),
            'predicted': round(float(pred[h]), 1),
        })

output = {
    'target': 'residual_load',
    'model': 'Temporal Fusion Transformer',
    'prediction_length_hours': prediction_length,
    'context_length_hours': context_length,
    'metrics': {
        'mae': round(float(tft_mae), 1),
        'rmse': round(float(tft_rmse), 1),
        'mape': round(float(tft_mape), 1),
    },
    'sample_forecast': sample_data,
}

with open('../dashboard/public/data/tft_residual.json', 'w') as f:
    json.dump(output, f, indent=2)

print('Saved tft_residual.json')
print(f"MAE:  {output['metrics']['mae']} MW")
print(f"RMSE: {output['metrics']['rmse']} MW")
print(f"MAPE: {output['metrics']['mape']}%")